In [3]:
%load_ext autoreload
%autoreload 2

import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from lib.models.latent_encoder import LatentEncoder
from lib.models.deepsdf import DeepSDF
from torchvision.transforms import v2
from lib.utils.config import load_config
from lib.data.metainfo import MetaInfo
import hydra
import glob
import cv2

from lib.utils.config import load_config
from lib.data.sampler import ChunkSampler
from lib.data.metainfo import MetaInfo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
cfg = load_config(
    "train_loss", ["+experiment/train_loss=latent_encoder_shapenet_chair_4096"]
)
metainfo = MetaInfo(cfg.data.data_dir, split="val_latent")
metainfo.load_snn()
sampler = ChunkSampler(metainfo.snn_labels, chunk_size=1)
max([i for i in sampler]), len([i for i in sampler])
loss_ckpt_path = "/home/borth/sketch2shape/logs/train_latent_encoder/runs/2024-02-15_22-31-15/checkpoints/epoch_019.ckpt"
shape_view_id = 11
shape_k = 16

cfg = load_config("optimize_sketch", ["+dataset=shapenet_chair_4096"])
metainfo = MetaInfo(cfg.data.data_dir)

cfg.loss_ckpt_path = loss_ckpt_path
cfg.model.shape_k = shape_k
cfg.model.shape_view_id = shape_view_id
cfg.model.shape_init = True
cfg.model.obj_id = metainfo.obj_ids[0]
model = hydra.utils.instantiate(cfg.model).to("cuda")

model.deepsdf.create_camera()

100%|██████████| 4096/4096 [00:19<00:00, 208.27it/s]


In [6]:
metainfo.load_snn()

In [8]:
metainfo._snn_data

,obj_id,image_id,label,image_type
0,52310bca00e6a3671201d487ecde379e,00000,0,sketch
1,52310bca00e6a3671201d487ecde379e,00000,0,normal
2,52310bca00e6a3671201d487ecde379e,00000,0,rendered_sketch
3,52310bca00e6a3671201d487ecde379e,00000,0,rendered_normal
4,52310bca00e6a3671201d487ecde379e,00001,0,sketch
...,...,...,...,...
1612795,6ade3daff4827379492d9da2668ec34c,00088,4479,rendered_normal
1612796,6ade3daff4827379492d9da2668ec34c,00089,4479,sketch
1612797,6ade3daff4827379492d9da2668ec34c,00089,4479,normal
1612798,6ade3daff4827379492d9da2668ec34c,00089,4479,rendered_sketch
